In [1]:
from qiskit import *
from qiskit.opflow import X, Y, Z, I, PauliTrotterEvolution
from qiskit.opflow import (StateFn, Zero, One, Plus, Minus, H,
                           DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn)
from qiskit.circuit import Parameter, Gate
from qiskit.quantum_info.operators import Operator, Pauli
import numpy as np
import math

In [2]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ionq import IonQProvider
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute
from qiskit import Aer
from qiskit.providers.aer import QasmSimulator

provider = IonQProvider(token = 'DvA5TJIIXwMEArPjI5SmGV2Ar9mUPMvo')

In [3]:
provider.backends()

[<IonQSimulatorBackend('ionq_simulator')>, <IonQQPUBackend('ionq_qpu')>]

In [4]:
backend = provider.get_backend("ionq_simulator")

In [5]:
f = 0.5
state2 = np.array([0,0,1,0])
state3 = np.array([0,0,0,1])
state22 = np.kron(state2,state2)
state33 = np.kron(state3,state3)
state22t = np.vstack(state22)
state33t = np.vstack(state33)
Hq = np.identity(16)-(1-f)*np.kron((state22 + state33)/math.sqrt(2),(state22t + state33t)/math.sqrt(2))
#Hq

In [6]:
import scipy

t = 0.1
e_iHqt = Operator(scipy.linalg.expm((0+1j)*Hq*t))
#e_iHqt

In [ ]:
controls = QuantumRegister(4)
circuit = QuantumCircuit(controls)
circuit.initialize([0, 1], 0)
circuit.initialize([0, 1], 1)
for i in range(50):
    circuit.unitary(e_iHqt, [0,1,2,3], label='e^iHqt')
#circuit.measure_all()
transpiled = transpile(circuit, backend)
#print(transpiled)
transpiled.measure_all()
#transpiled.draw()


In [ ]:


job_sim = backend.run(transpiled, shots=50)
result_sim = job_sim.result()
counts = result_sim.get_counts()
counts


In [ ]:

transpiled.save_statevector()
backend = QasmSimulator()
backend_options = {'method': 'statevector'}
job = execute(transpiled, backend, backend_options=backend_options)
job_result = job.result()
print(job_result.get_statevector(transpiled))